# Train Linear Gaussian

In [1]:
using AutomotiveDrivingModels

## Parameters

In [2]:
include(Pkg.dir("AutomotiveDrivingModels", "scripts", "extract_params.jl"))
nothing

In [3]:
dset_filepath_modifier = "_following" # _freeflow, _following, _lanechange

"_following"

In [4]:
dset = JLD.load(joinpath(EVALUATION_DIR, "dataset2" * dset_filepath_modifier * ".jld"),
                "model_training_data")::ModelTrainingData2
runlogs_original = load_runlogs(dset)
streetnets = load_streetnets(runlogs_original)

cv_split_outer = get_fold_assignment_across_drives(dset, N_FOLDS)
cv_split_inner = drop_fold!(deepcopy(cv_split_outer), 1)
@assert(cv_split_inner.nfolds > 0)

nframes = nrow(dset.dataframe)
ntraces = length(cv_split_outer.seg_assignment);

runlog_id_ntraces: 

## optimize hyperparams

In [5]:
train_def = BehaviorTrainDefinition(
    LG_TrainParams(indicators=INDICATOR_SET2),
    [
        BehaviorParameter(:ridge_regression_constant, [0.0,0.5,1.0,2.0,10.0,100.0,1000.0], 5)
    ])
preallocated_data = preallocate_learning_data(dset, train_def.trainparams)
print(train_def); sleep(0.1)

[1,4,2,4,2,1,1,6,5,8,34,21,96,77,69,39,8,39,82]
id_to_fold: [1,5,1,2,5,3,5,5,2,5,4,3,1,3,4,5,2,5,2]
499
BehaviorTrainDefinition
targets:    (:f_des_angle,:f_accel)
indicators: [:posFyaw,:posFt,:speed,:velBx,:velBy,:velFs,:velFt,:turnrate,:acc,:accFs,:accFt,:accBx,:accBy,:d_ml,:d_mr,:ttcr_ml,:ttcr_mr,:est_ttcr,:a_req_stayinlane,:n_lane_left,:n_lane_right,:has_lane_right,:has_lane_left,:curvature]
λ:          0.5

hyperparams:
	ridge_regression_constant: [0.0, 0.5, 1.0, 2.0, >10.0<, 100.0, 1000.0]


In [6]:
AutomotiveDrivingModels.optimize_hyperparams_cyclic_coordinate_ascent!(
                    train_def, dset, preallocated_data, cv_split_inner)
print(train_def); sleep(0.1)

                                       λ   i   logl  logl*
optimize_hyperparams_cyclic_coordinate_ascent iteration 1  et: 0.0007920265197753906
               ridge_regression_constant   1 4.9711   -Inf
               ridge_regression_constant   2 4.9712 4.9711
               ridge_regression_constant   3 4.9712 4.9712
               ridge_regression_constant   4 4.9712 4.9712
               ridge_regression_constant   5 4.9707 4.9712
               ridge_regression_constant   6 4.8850 4.9712
               ridge_regression_constant   7 3.7285 4.9712
optimize_hyperparams_cyclic_coordinate_ascent iteration 2  et: 28.9803569316864
BehaviorTrainDefinition
targets:    (:f_des_angle,:f_accel)
indicators: [:posFyaw,:posFt,:speed,:velBx,:velBy,:velFs,:velFt,:turnrate,:acc,:accFs,:accFt,:accBx,:accBy,:d_ml,:d_mr,:ttcr_ml,:ttcr_mr,:est_ttcr,:a_req_stayinlane,:n_lane_left,:n_lane_right,:has_lane_right,:has_lane_left,:curvature]
λ:          2.0

hyperparams:
	ridge_regression_constant: [0.0, 0.5,

## train

In [9]:
LG = train(dset, preallocated_data, train_def.trainparams, 1, cv_split_outer, true);
print(LG)

LG
	indicator: acc
	A: 
		[    0.001199,    -0.004475]
		[   -0.002327,    -0.994266]
	μ∞:
		[    0.000000,     0.000000]
	Σ:
		[    0.000124,    -0.000015]
		[   -0.000015,     0.000544]


227.9411764705882